# Introducing Pandas Objects

At the very basic level, Pandas objects can be thought of as enhanced versions of NumPy structured arrays in which the rows and columns are identified with labels rather than simple integer indices.
As we will see during the course of this chapter, Pandas provides a host of useful tools, methods, and functionality on top of the basic data structures, but nearly everything that follows will require an understanding of what these structures are.
Thus, before we go any further, let's introduce these three fundamental Pandas data structures: the ``Series``, ``DataFrame``, and ``Index``.

We will start our code sessions with the standard NumPy and Pandas imports:

In [ ]:
import tempfile

import numpy as np
import pandas as pd

## The Pandas Series Object

A Pandas ``Series`` is a one-dimensional array of indexed data.
It can be created from a list or array as follows:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
data

As we see in the output, the ``Series`` wraps both a sequence of values and a sequence of indices, which we can access with the ``values`` and ``index`` attributes.
The ``values`` are simply a familiar NumPy array:

In [ ]:
data.values

The ``index`` is an array-like object of type ``pd.Index``, which we'll discuss in more detail momentarily.

In [ ]:
data.index

Like with a NumPy array, data can be accessed by the associated index via the familiar Python square-bracket notation:

In [ ]:
data[1]

In [ ]:
data[1:3]

As we will see, though, the Pandas ``Series`` is much more general and flexible than the one-dimensional NumPy array that it emulates.

### ``Series`` as generalized NumPy array

From what we've seen so far, it may look like the ``Series`` object is basically interchangeable with a one-dimensional NumPy array.
The essential difference is the presence of the index: while the Numpy Array has an *implicitly defined* integer index used to access the values, the Pandas ``Series`` has an *explicitly defined* index associated with the values (the former can also be used using the ``iloc`` method, the latter using the ``loc`` method).

This explicit index definition gives the ``Series`` object additional capabilities. For example, the index need not be an integer, but can consist of values of any desired type.
For example, if we wish, we can use strings as an index:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

And the item access works as expected:

In [ ]:
data['b']

We can even use non-contiguous or non-sequential indices:

In [ ]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=[2, 5, 3, 7])
data

In [ ]:
data[5]

### Series as specialized dictionary

In this way, you can think of a Pandas ``Series`` a bit like a specialization of a Python dictionary.
A dictionary is a structure that maps arbitrary keys to a set of arbitrary values, and a ``Series`` is a structure which maps typed keys to a set of typed values.
This typing is important: just as the type-specific compiled code behind a NumPy array makes it more efficient than a Python list for certain operations, the type information of a Pandas ``Series`` makes it much more efficient than Python dictionaries for certain operations.

The ``Series``-as-dictionary analogy can be made even more clear by constructing a ``Series`` object directly from a Python dictionary:

In [ ]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

By default, a ``Series`` will be created where the index is drawn from the sorted keys.
From here, typical dictionary-style item access can be performed:

In [ ]:
population['California']

Unlike a dictionary, though, the ``Series`` also supports array-style operations such as slicing:

In [ ]:
population['California':'New York']

We'll discuss some of the quirks of Pandas indexing and slicing in [Data Indexing and Selection](pandas_data_manipulation.ipynb).

## The Pandas DataFrame Object

The next fundamental structure in Pandas is the ``DataFrame``.
Like the ``Series`` object discussed in the previous section, the ``DataFrame`` can be thought of as a generalization of a NumPy array.

### DataFrame as a generalized NumPy array
If a ``Series`` is an analog of a one-dimensional array with flexible indices, a ``DataFrame`` is an analog of a two-dimensional array with both flexible row indices and flexible column names.
Just as you might think of a two-dimensional array as an ordered sequence of aligned one-dimensional columns, you can think of a ``DataFrame`` as a sequence of aligned ``Series`` objects.
Here, by "aligned" we mean that they share the same index.

To demonstrate this, let's first construct a new ``Series`` listing the area of each of the five states discussed in the previous section:

In [ ]:
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)
area

Now that we have this along with the ``population`` Series from before, we can use a dictionary to construct a single two-dimensional object containing this information:

In [ ]:
states = pd.DataFrame({'population': population,
                       'area': area})
states

Like the ``Series`` object, the ``DataFrame`` has an ``index`` attribute that gives access to the index labels:

In [ ]:
states.index

Additionally, the ``DataFrame`` has a ``columns`` attribute, which is an ``Index`` object holding the column labels:

In [ ]:
states.columns

Thus the ``DataFrame`` can be thought of as a generalization of a two-dimensional NumPy array, where both the rows and columns have a generalized index for accessing the data.

### Constructing DataFrame objects

A Pandas ``DataFrame`` can be constructed in a variety of ways.
Here we'll give two examples.

#### From a dictionary of Series or NumPy arrays

As we saw before, a ``DataFrame`` can be constructed from a dictionary of ``Series`` or NumPy arrays. Note that this method copies (by default) the data, so you can safely modify the output without affecting the original:

In [ ]:
df = pd.DataFrame({'population': population,
              'area': area})
df

In [ ]:
area["Illinois"] = 999  # won't affect df

In [ ]:
df

To create a df _without_ copying the data, one can use the `copy=False` argument:

In [ ]:
df_byref = pd.DataFrame({'population': population,
                   'area': area}, copy=False)
df_byref

In [ ]:
area["Illinois"] = 555555  # will affect df, remember, it was 999 before
df

#### From a two-dimensional NumPy array

Given a two-dimensional array of data, we can create a ``DataFrame`` with any specified column and index names.
If omitted, an integer index will be used for each:

In [ ]:
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

### To and from disk files

Pandas supports a range of file formats, including CSV, Excel, HDF5, and SQL to load and save data to disk.

In [ ]:
# Writing to a CSV file
tmpfilename = tempfile.NamedTemporaryFile().name
states.to_csv(tmpfilename)

In [ ]:
# Reading from a CSV file
states_loaded = pd.read_csv(tmpfilename, index_col=0)
states_loaded

In [ ]:
# Writing to an SQL database
import sqlite3
conn = sqlite3.connect(tempfile.NamedTemporaryFile().name)  # create a temporary database
states.to_sql('states', conn)

In [ ]:
# Reading from an SQL database
states_loaded = pd.read_sql('select * from states', conn, index_col='index')
states_loaded

#### Bonus: ROOT files

In the HEP community, ROOT files are a common format to store data. Pandas can read and write ROOT files using the [`uproot` package](https://uproot.readthedocs.io/en/latest/).



In [ ]:
import uproot

# Writing to a ROOT file
tmpfilename = tempfile.NamedTemporaryFile().name
with uproot.recreate(tmpfilename) as f:
    f["states"] = df.reset_index()  # we cannot store string index in ROOT, they are meant to be ordered

In [ ]:
# Reading from a ROOT file
with uproot.open(tmpfilename) as f:
    states_loaded = f["states"].arrays(("population", "area"), library="pd")
states_loaded